# Reporte de implementación del método de eliminación por bloques y Algoritmo One-Sided Jacobi para SVD

**Fecha**

19 de Abril de 2020

## 0. Introducción 

Al tratar de encontrar la solución de sistemas lineales $Ax=B$ de tamaño considerable, una estrategia interesante es recurrir a métodos que intercambien el problema original por hallar la respuesta de una serie de sistemas de ecuaciones de menor dimensión, que puedan ser resueltos con otras técnicas y cuyas soluciones permitan armar la solución del sistema que originalmente era de interés. 

Esta es la ídea en la que se basa el método de eliminación por bloques, en el cual un sistema lineal se descompone a través de bloques los cuales determinan a su vez otros sistemas lineales más pequeños, en los que aparece el *complemento de Schur*.

![bloques](../images/bloques.png)


Por otro lado, la descomposición en valores singulares de una matriz (SVD por sus siglas en inglés) consiste hallar matrices que permitan su factorización; en concreto, se refiere a hallar dos matrices ortonormales U y V asociadas al dominio y co-dominio de $A$, junto con una matriz diagonal $\Sigma$ cuyas entradas se encuentra determinadas por los **valores singulares** de la matriz en cuestión. 

Para hallar la solución al sistema $Ax=b$, dicha estructura $U$, $V$ y $\Sigma$ de la descomposición SVD para replantear el sistema original, en otro más sencillo que se puede resolver con sustitución atrás/adelante, para posteriormente recuperar la solución de interés con una multiplicación *matriz-vector*.

Así pues, el método de eliminación por bloques, se puede echar mano de la factorización SVD para dar solución a los sistemas lineales de menor dimensión que resultan de re-expresar el problema en términos de sus correspondientes bloques.

Estas ideas fueron son exploradas a continuación para llevar a cabo la implementación del método de eliminación por bloques, a través del lenguaje de programación R. Es así que en este tenor se planteó como meta consolidar los siguientes:

**Objetivos**

Realizar la implementación mediante el lenguaje de programación R de:

1) El algoritmo One-Sided Jacobi para aproximar la descomposición SVD de una matriz,

2) Un solver de sistemas lineales a partir de la descomposición SVD de la matriz del sistema,

3) El método de eliminación por bloques para sistemas lineales usando 1) y 2).

**Especificaciones de ambiente común de trabajo**

Cabe destacar que para consolidar un entorno común de trabajo se empleo la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

1. Este reporte incluye la consideraciones y sugerencias realizadas por el Equipo de Revisión (E-Rev), que se encuentras disponibles para su consulta a través del siguiente vínculo [reportes de revisión](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/tree/master/test).
2. En complemento, a partir de las implementaciones aquí mostradas se ha redactado un [reporte ejecutivo de resultados](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/results/Reporte_resultados.ipynb) disponible para su consulta.


Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Aspectos teóricos

### 1.1 Descomposición SVD

Por resultados de álgebra lineal y análisis matemático, se sabe que para una matriz ![$A \in \mathbb{R}^{mxn}$](https://render.githubusercontent.com/render/math?math=A%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxn%7D&mode=inline) es posible encontrar una factorización de ésta en términos de matrices ![$U \in \mathbb{R}^{mxm}, V \in \mathbb{R}^{nxn}$](https://render.githubusercontent.com/render/math?math=U%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxm%7D%2C%20V%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bnxn%7D&mode=inline) ortogonales tales que: ![$A = U\Sigma V^T$](https://render.githubusercontent.com/render/math?math=A%20%3D%20U%5CSigma%20V%5ET&mode=inline) con ![$\Sigma = diag(\sigma_1, \sigma_2, \dots, \sigma_p) \in \mathbb{R}^{mxn}$](https://render.githubusercontent.com/render/math?math=%5CSigma%20%3D%20diag%28%5Csigma_1%2C%20%5Csigma_2%2C%20%5Cdots%2C%20%5Csigma_p%29%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxn%7D&mode=inline), ![$p = \min\{m,n\}$](https://render.githubusercontent.com/render/math?math=p%20%3D%20%5Cmin%5C%7Bm%2Cn%5C%7D&mode=inline) y ![$\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_p \geq 0$](https://render.githubusercontent.com/render/math?math=%5Csigma_1%20%5Cgeq%20%5Csigma_2%20%5Cgeq%20%5Cdots%20%5Cgeq%20%5Csigma_p%20%5Cgeq%200&mode=inline).

Típicamente $U$, $\Sigma$ y $V$ se suelen asociar a acciones de $A$ sobre la bola unitaria y la imagen de ésta sobre $A$.

Es importante que existen representaciones alternas dicha factorización, pero similarmente tales se basan en encontrar matrices ortogonales asociadas al dominio y condominio de $A$, junto con sus valores singulares. Véase https://en.wikipedia.org/wiki/Singular_value_decomposition#Reduced_SVDs

### 1.2.1 Algoritmo One-Sided Jacobi para aproximar la descomposición SVD

**Nota:** Los detalles del algoritmo a continuación se han tomado de la nota https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/III.computo_matricial/3.3.d.SVD.ipynb

Para obtener versiones numéricas de la descomposición SVD de una matriz existe un método iterativo conocido como el **algoritmo One-Sided Jacobi** que se basa en la aplicación sucesiva de rotaciones de Givens, para ortogonalizar las columnas de las matrices involucradas, así como estimar los respectivos valores singulares. En concreto, la idea en que se basa tal método es multiplicar a la matriz ![$A \in \mathbb{R}^{m \times n}$](https://render.githubusercontent.com/render/math?math=A%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bm%20%5Ctimes%20n%7D&mode=inline) por la derecha de forma repetida por matrices ortogonales de nombre **rotaciones Givens** hasta que se converja a ![$U \Sigma$](https://render.githubusercontent.com/render/math?math=U%20%5CSigma&mode=inline).


Es decir, se trata de el algoritmo que realiza interacciones sobre las matrices $A,V$ de la forma: $A^{(k+1)} = A^{(k)}U^{(k)}, V^{(k+1)} = V^{(k)}U^{(k)}$ para $k>0$ donde: $U^{(k)}$ son matrices de rotación del plano $(i,j)$. Esto es, una identidad matriz identidad compatible con las dimensiones de $A$ y $V$ pero que actúa únicamente sobre las entradas dadas los índices $i$ y $j$ a través de las fórmulas que determinan una rotación de Givens

$$u_{ii}^{(k)} = cos(\theta), \quad u_{ij}^{(k)} = sen(\theta)$$

$$u_{ji}^{(k)}=-sen(\theta), \quad u_{jj}^{(k)}=cos(\theta).$$


Es decir, la multiplicación $A^{(k)}U^{(k)}$ sólo involucra a $2$ columnas de $A^{(k)}$:

$$(a_i^{(k+1)} a_j^{(k+1)}) = (a_i^{(k)} a_j^{(k)})\left[ \begin{array}{cc}
cos(\theta) & sen(\theta)\\
-sen(\theta) & cos(\theta)
\end{array}
\right]$$


Al realizarse rotaciones con referencias a las entradas $(i,j)$ de $A^{(k)}$, en el proceso iterativo debe existir una secuencia, denominada *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n-1)}{2}$ rotaciones a aplicarse y es dependiente de cuántas columnas son o no ortogonales dentro del proceso (iterativo), recordando que en cada *sweep* se ortogonalizan $2$ columnas empleando una rotación de Givens.

Al finalizar el algoritmo los valores singulares calculados son las normas Euclidianas de cada columna de $A^{(k)}$. Las columnas normalizadas de $A^{(k)}$ aproximarán a las columnas de $U$. 

**Notas:**


* El ángulo $\theta$ de la rotación de Givens en cada sweep se debe eligir de acuerdo a $2$ criterios:

    1) El ángulo es $0$ si $a_i^{T(k)}a_j^{(k)}=0$ y por lo tanto las columnas $i,j$ son ortogonales y no se hace rotación.
    
    2) $\theta \in (\frac{-\pi}{4}, \frac{pi}{4})$ tal que $a_i^{T(k+1)}a_j^{(k+1)}=0$ y para este caso se calculan $\xi, t, cs,sn$ (las variables $cs, sn$ hacen referencia a $cos(\theta), sen(\theta)$.
* La convergencia del algoritmo depende del ángulo de rotación, pero se saber que si los *sweep*  se realizan secuencialmente siguiente el ordenamiento cíclico por renglones considerando pares de columnas $(1,2), (1,3), \dots, (1,n), (2,3), (2,4), \dots, (n-1,n)$, tal ordenamiento si $|\theta| \leq \frac{\pi}{4}$.

**Consideraciones sobre la implementación:**
* El número de *sweeps* máximos a realizarse se debe controlarse en la implementación,
* El número de columnas ortogonales también es un parámetro a controlar en el algoritmo,
* La implementación requerirá entonces el desarrollo de funciones capaces de:
 * Determinar si dos vectores (columnas) son ortogonales dada cierta tolerancia,
 * Un mecanismo para generar la secuencia de índices $(1,2), (1,3), \dots, (1,n), (2,3), (2,4), \dots, (n-1,n)$ en la que deben correr los *sweeps*,
 * Calcular el valor de la función signo sobre números reales, puesto que se involucra en la expresión de las rotaciones de Givens,


De todos lo anterior, el Algoritmo One-Sided Jacobi se puede describir como sigue:

**Algoritmo One-Sided Jacobi**

Entrada: 

* matriz $A \in \mathbb{R}^{m \times n}$: matriz a la que se le calculará su SVD.

* $TOL$ controla la convergencia del método.

* $maxsweeps$ número máximo de sweeps (descritos en los comentarios de abajo).

Salida: 

* matrices $V \in \mathbb{R}^{n \times n}$ ortogonal, $W \in \mathbb{R}^{m \times n}$ representada en el algoritmo por $A^{(k)}$ para un valor de $k$ controlado por la convergencia (descrita en los comentarios de abajo).


Nota: se utilizará la notación $A^{(k)}=[a_1^{(k)} a_2^{(k)} \cdots a_n^{(k)}]$ con cada $a_i^{(k)}$ como $i$-ésima columna de $A$ y $k$ representa el *sweep*.

Definir $A^{(0)}=A$, $V^{(0)}=I_n$ (*sweep* $=0$). 

* Mientras no se haya llegado al número máximo de sweeps ($k \leq maxsweeps$) o el número de columnas ortogonales sea menor a $\frac{n(n-1)}{2}$:

    * Para todos los pares con índices $i<j$ generados con alguna metodología (descrita en la sección de abajo) y para k desde 0 hasta convergencia:
    
        * Revisar si las columnas $a_i^{(k)}, a_j^{(k)}$ de $A^{(k)}$ son ortogonales (el chequeo se describe en los comentarios). Si son ortogonales se incrementa por uno la variable $num\text{_}columnas\text{_}ortogonales$. Si no son ortogonales:
        
            * Calcular $\left[ \begin{array}{cc}
a & c\\
c & b
\end{array}
\right]$ la submatriz $(i,j)$ de $A^{T(0)}A^{(0)}$ donde: $a = ||a_i^{(k)}||_2^2, b=||a_j^{(k)}||_2^2, c=a_i^{T(k)}a_j^{(k)}$. 

            * Calcular la rotación Givens que diagonaliza $\left[ \begin{array}{cc}
a & c\\
c & b
\end{array}
\right]$ definiendo: $\xi = \frac{b-a}{2c}, t=\frac{signo(\xi)}{|\xi| + \sqrt{1+\xi^2}}, cs = \frac{1}{\sqrt{1+t^2}}, sn = cs*t$. Recordar que $signo(a) = \begin{cases}
1 &\text{ si } a \geq 0 ,\\
-1 &\text{ si } a < 0
\end{cases}$.
    
            * Actualizar las columnas $i,j$ de $A^{(k)}$. Para $\ell$ de $1$ a $n$:
    
                * $temp = A^{(k)}_{\ell i}$
        
                * $A_{\ell i}^{(k)} = cs*temp - sn*A_{\ell j}^{(k)}$
        
                * $A_{\ell j}^{(k)} = sn*temp + cs*A_{\ell j}^{(k)}$
        
            * Actualizar a la matriz $V^{(k)}$. Para $\ell$ de $1$ a $n$:
    
                * $temp = V_{\ell i}^{(k)}$
        
                * $V_{\ell i}^{(k)} = cs*temp - sn*V_{\ell j}^{(k)}$
        
                * $V_{\ell j}^{(k)} = sn*temp + cs*V_{\ell j}^{(k)}$
                
    * Incrementar por uno la variable $k$ que cuenta el número de sweeps.


### 1.2.2 Solución de sistemas lineales a partir de la descomposición de la matriz del sistema,

Si conocemos la descomposición SVD de una matriz $A$, podemos resolver un sistema lineal $Ax=b$ como sigue

a) **Descomposición SVD:** Estimar factores ![$U, \Sigma, V$](https://render.githubusercontent.com/render/math?math=U%2C%20%5CSigma%2C%20V&mode=inline) tales que ![$A=U \Sigma V^T$](https://render.githubusercontent.com/render/math?math=A%3DU%20%5CSigma%20V%5ET&mode=inline).

b) **Solución de sistema intermedio:** resolver el sistema diagonal ![$\Sigma d = U^Tb$](https://render.githubusercontent.com/render/math?math=%5CSigma%20d%20%3D%20U%5ETb&mode=inline). Este paso se puede hacer con solución hacia delante o atrás, puesto que es un sistema diagonal.

c) **Armado de solución de sistema original:** hacer la multiplicación matriz vector para obtener ![$x$](https://render.githubusercontent.com/render/math?math=x&mode=inline): ![$x=Vd$](https://render.githubusercontent.com/render/math?math=x%3DVd&mode=inline).

De las exposición anterior, se desprende que si queremos resolver numéricamente un sistema $Ax=b$ se puede emplear el algoritmo One-Sided Jacobi para aproximar la descomposición SVD y posteriormente emplear el proceso descrito en el apartado 1.2

**Consideraciones sobre la implementación**

* Se requiere un mecanismo para que dada una matriz $A$ se pueda aproximar su descomposición $SVD$, como el descrito en el apartado 1.2.1

### 1.4 Método de eliminación por bloques para resolver sistemas lineales

Considérese un sistema de la forma: $Ax=b$ escrito como:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right] \cdot 
\left[\begin{array}{c}
x_1\\
x_2
\end{array}
\right] =
\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]
\end{array}
$$

con $x_1 \in \mathbb{R}^{n_1}, x_2 \in \mathbb{R}^{n_2}, A_{11} \in \mathbb{R}^{n_1 \times n_1}, A_{22} \in \mathbb{R}^{n_2 \times n_2}, b_1 \in \mathbb{R}^{n_1}, b_2 \in \mathbb{R}^{n_2}$.

El método de eliminación por bloque consiste en que, dado el caso de que $A$ y $A_{11}$ sean invertibles (no singulares), las ecuaciones inducidas por los bloques se pueden resolver como sigue:

1. Si conocemos $x_2$, entonces de la primera ecuación de bloques![A_{11}x_1 + A_{12}x_2 = b_1](https://render.githubusercontent.com/render/math?math=A_%7B11%7Dx_1%20%2B%20A_%7B12%7Dx_2%20%3D%20b_1)podemos obtener $x_1$ al resolver el sistema $A_{11} x_1 = b_1-A_{12}x_2$ o de modo equivalente $x_1 = A_{11}^{-1}(b_1-A_{12}x_2)$, que es un sistema lineal de menores dimensiones.
2. Por otro lado, haciendo los cálculos, de la segunda ecuación por bloques se sigue que $S x_2 = b_2 - A_{21}A_{11}^{-1}b_1$, en donde a) $S := A_{22}-A_{21}A_{11}^{-1}A_{12}$ se conoce como el **complemento de Schur** del bloque $A_{11}$ en $A$, y b) $S$ es no singular si y sólo si $A$ es no singular. Así para estimar $x_2$, basta resolver $S x_2 = b_2 - A_{21}A_{11}^{-1}b_1$ que es un sistema más pequeño.
3. La solución del sistema $Ax=b$ se obtiene en términos de $x_1$ y $x_2$.

Con este procedimiento reducimos el problema de hallar la solución de un sistema $Ax = b$, pero debemos contar con un método que permita resolve los sistemas lineales más pequeños que se describen en los numerales 1. y 2.

**Consideraciones sobre la implementación**

* Se requiere un mecanismo para dividir $A$ en bloques de ciertas dimensiones,
* Se requiere de un procedimiento para resolver sistema lineales asociados, con dimensiones menores. En estos se puede explotar los algoritmos previamente descritos que abordan la descomposición SVD.

**Algoritmo de Eliminación por bloques**

Sean $A$ y $A_{11}$ no singulares.

0) Calcular la división de $A$ en bloques $A_{11}$, $A_{12}$, $A_{21}$ y $A_{22}$.
1) Calcular $A_{11}^{-1}A_{12}$ y $A_{11}^{-1}b_1$, lo cual equivale a resolver por algún método a los sistemas de ecuaciones lineales: $$A_{11}Y=A_{12}$$ y $$A_{11}y=b_1$$
2) Calcular el complemento de Schur del bloque $A_{11}$ en $A$: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. 
3) Calcular $ \hat{b} = b_2-A_{21}A_{11}^{-1}b_1$.

3) Resolver $Sx_2 = \hat{b}$.

4) Resolver $A_{11}x_1 = b_1-A_{12}x_2$.
5) Formar $x:=[x_1 x_2]^T$, que es la solución del sistema

### 1.5 Organización del proyecto derivada de los aspectos teóricos

De la revisón teórica recien hecha, se identificó que la implementación requiere contemplar diferentes hitos. La organización de estos a través del proyecto se resume a continuación:

|   | Tema                                                                         | Rubro                                                                                                                 |                                           Usada para                                          | Sección teórica | Sección de implementación |
|:-:|------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------:|-----------------|---------------------------|
| 1 | Algoritmo One-Sided Jacobi                                                   | Generación de índices                                                                                                 |                        Secuencia de índices para ortogonalizar columnas                       | 1.2.1           | 2.1                       |
| 2 | Algoritmo One-Sided Jacobi                                                   | Ortogonalidad de vectores                                                                                             |          Determinar numéricamente si dos vectores son ortogonales dada una tolerancia         | 1.2.1           | 2.2                       |
| 3 | Algoritmo One-Sided Jacobi                                                   | Signo de un número                                                                                                    |                         Cálculo de coeficientes en rotaciones de Gives                        | 1.2.1           | 2.3                       |
| 4 | Algoritmo One-Sided Jacobi                                                   | Implementación del algoritmo                                                                                          | Obtener aproximaciones numéricas de la descomposición SVD de una matriz                       |   1.2.1         | 3.1                       |
| 5 | Respuesta a un sistema lineal usando descomposición SVD de matriz de sistema | Implementación del algoritmo                                                                                          |      Estructura de como resolver un sistema lineal una vez conocida la descomposición SVD     |      1.2.2      | 2.4                       |
| 6 | Respuesta a un sistema lineal usando descomposición SVD de matriz de sistema | Solver dadas matrices de cierta estructura ($U$, $\Sigma$ y $V$)                                                      | Estructura de como resolver un sistema lineal una vez conocida la descomposición SVD          |      1.2.2      | 2.4                       |
| 7 | Respuesta a un sistema lineal usando descomposición SVD de matriz de sistema | Solver dadas matrices de descomposición SVD                                                                           | Solución de un sistema linea usando descomposición SVD                                        |       1.2.3     | 3.2                       |
| 8 | Método de eliminación por bloques usando SVD                                 | Creación de bloques                                                                                                   | Se refiere a un procedimiento para dividir la matriz y vector de un sistema lineal en bloques |       1.2.3    | 3.3.1                     |
| 9 | Método de eliminación por bloques usando SVD                                 | Solver de sistemas lineales usando método de eliminación por bloques y solver basado en Algoritmo de Jacobi One-Sided |   Resolver un sistema lineal por el método de eliminación por bloques usando la descomposición SVD para resolver sistemas lineales de menor dimensión  |     1.2.3       | 3.3.2                     |

Para abordar tales puntos, los mecanismos anteriores se dividieron primero en un bloque de funciones en R que menor complejidad, correspondiente al apartaod **2. Funciones auxiliares** y un posterior que aborda los mecanismos más complejos **3. Algoritmo SVD y solución de sistema lineal**, cuya exposición se realiza a continuación.


## 2. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 Generación de índices

**Propósito:**

Dado un número entero positivo $n$, generar una función *indices* que devuelva una lista de índices o sublistas  siguiente:

$$\{(1,2), (1,3), \ldots (1,n)\} \cup  \{(2,3), (2,4), \ldots (2,n)\} \cup  \{(3,4),\ldots (2,n)\} \cup \{(n-1,n)\} $$

**Notas:** 

* La cardinalidad del conjunto de índices es 

$$\frac{n(n+1)}{2}$$

* La función debe regresar elementos un elemento que se pueda recorrer y que a su vez se puedan acceder a los elementos de las tuplas. Por ejemplo, si contiene a la tupla (1,3), queremos poder acceder a 1 y 3 posteriormente en la salida de la función.

* Es relevante destacar que tras realizar perfilamiento, se identificó que la implementación pleanteada en un inició perjudicaba el perfomance del proyecto, sin embargo se dejó presente para ejemplificar los cambios que sufrió al proyecto a partir del anáilis hecho por el **Equipo de Revisión**.

**Funciones del proyecto  que depende:**

Ninguna.

**Reporte de revisión:**

Cabe destacar que el reporte de revisión correspondiente a esta función se encuentra disponible para su consulta en este [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_FuncionSigno.ipynb)


In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n (int): número entero postivo 
  #       se refiere al número de columnas de una matriz
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices
}

### 2.2 Verificación de ortogonalidad entre vectores

**Propósito:**

Dados dos vectores $u,v\in \mathbb{R}^n$ y un parámetro $TOL$ de tolerancia, generar una función *ortogonal* que verifique si el producto punto de la versión normalizada de ambos vectores se considera ortogonal; es decir, que evalue la verdad o falsedad de la afirmación:

$$ \frac{u \cdot v^T}{|| u ||_2 || v ||_2 } < TOL$$

**Notas:** 

* $TOL$ debe ser un parámetro (número real positivo) de este funcion que el usuario pueda definiar a voluntad,

* La norma de los vectores a involucrarse en la definición es la norma-2,
* La función debe regresar un valor booleano, que nos diga si dada la tolerancia la versión normalizada de ambos vectores se considera ortogonal.

**Funciones del proyecto  que depende:**

Ninguna.

**Reporte de revisión:**

El reporte de revisión correspondiente a esta función se encuentra disponible para su consulta en este [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_VerifOrtogonalidad.ipynb)

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, de acuerdo a cierto nivel de tolerancia, 
  # arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
  #   u (vector): vector de dimension n,
  #   v (vector): vector de dimension n, 
  #   TOL (numeric): real positivo, que sirve como parametro de tolerancia para evaluar ortogonalidad de u y v. 
  #   Notas: 
  #   1) Valor por default TOL es 10^-8
  #   2) Se sugiere una TOL mayor a 10^-32.
  # Returns: 
  #   Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){ret<-0} 
    else{ 
        if( (u%*%v)/(norm(u,type ="2")*norm(v,type ="2")) < TOL){ret<-1}
        else{ret<-0}  
    }
  ret
}

### 2.3 Función signo

**Propósito:**

Generar una función *signo* que verifique el signo de un número real; es decir:

$$signo(a) = \begin{cases} 1 & \mbox{ si } a \geq 0 \\ -1 & \mbox{ si } a < 0 \end{cases}$$

**Funciones del proyecto  que depende:**

Ninguna.

**Reporte de revisión:**

El reporte de revisión correspondiente a esta función se encuentra disponible para su consulta en este [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_FuncionSigno.ipynb)


In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

### 2.4 Solver dada descomposición SVD

**Propósito:**

Esta función asumirá que dada una matriz $ A \in \mathbb{R}^{m \times n}$, se conoce su descomposición SVD, de acuerdo a las consabidas matrices $U \in \mathbb{R}^{m \times m}$, $\Sigma\in \mathbb{R}^{m \times n}$ y $V \in \mathbb{R}^{n \times n}$, así como un vector $b \in \mathbb{R}^m$. La idea será formar una función *solver* que devuelva la solución el sistema lineal $Ax=b$ a través del siguiente procediemiento:

* Resolver el sistema de ecuaciones $$ \Sigma d = U^T b$$, empleando alguna de las librerías [backsolve/forwardsolve](https://stat.ethz.ch/R-manual/R-devel/library/base/html/backsolve.html) de R.
* Formar al vector $x:=Vd$ la cual será solución del sistema $Ax=b$.

**Notas:** 

* La matrices $U$, $\Sigma$, $V$ deben ser parte de las entradas la función.

**Funciones del proyecto  que depende:**

Ninguna.

**Reporte de revisión:**

El reporte de revisión correspondiente a esta función se encuentra disponible para su consulta en este [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_Solver_SVD.ipynb)


In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, y construye x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #     U (matriz): matriz para lado derecho de sistema S d = U^Tb, con entrada reales y dimension m x n,
    #     S (matriz): matriz diagonal, que define sistema sistema S d = U^Tb, con entrada reales y dimension n x n,
    #     V (matriz): para construir x, con entrada reales y dimension n x n, 
    #     b (vector): vector con el que se forma lado derecho de primer sistema, de dimension m.
    # Returns: 
    #     x (vector): vector formado por la solucion de S d = U^Tb, multiplicado por V, con dimension n
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

## 3. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 3.1 One-sided Jacobi numerical aproximación

**Propósito:**

Dada una matriz $A \in \mathbb{R}^{ m \times n}$, crear una función *SVD_jacobi_aprox* que permita obtener aproximaciones numéricas asociadas a la descomposición SVD de esta,. En concreto, si la descomposición de $A$ está dada por las consabidas matrices $U \in \mathbb{R}^{m \times m}$, $\Sigma\in \mathbb{R}^{n \times n}$ y $V \in \mathbb{R}^{n \times n}$,  queremos obtener versiones numéricas de 1) $V$ y 2) $W:=U \Sigma$.

Ello se debe realizar con base en las siguientes especificaciones, que circunscriben el Algoritmo One-sided Jacobi para la descomposición SVD:

**Entradas:**

* $$A \in \mathbb{R}^{ m \times n}$$,
* TOL (numero real, parámetro de tolerancia prefijado en $10^{-8}$),
* *maxsweeps* (entero mayor a cero). **Nota:** Las rotaciones que se aplicarán sucesivamente para ortogonalizar las matrices a partir de A se realizan en una secuencia con nombre *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n+1)}{2}$ rotaciones (pues depende de cuántas columnas son o no ortogonales) y en cada *sweep* se ortogonalizan 2 columnas. El número de *sweeps* a realizar se controla con esta variable.

**Salidas:**

* $V \in \mathbb{R}^{n \times n}$ (dadas por $V\leftarrow V^k$, al final del proceso iterativo),
* $S \in \mathbb{R}^{n \times n}$ (dada por norma de $A^k$, al final del proceso iterativo),
* $U \in \mathbb{R}^{m \times n}$ (dada por $U\leftarrow A^k$ a la que se le han normalizado las columnas, al final del proceso iterativo),

**Funciones del proyecto  que depende:**

Signo y ortongonal (véanse secciones 2.2 y 2.3)

**Reporte de revisión:**

Los reportes de revisión correspondiente a esta función se encuentra disponible para su consulta en el [vínculo 1](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_One-sided_Jacobi.ipynb) y [vínculo 2](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_One-sided_Jacobi_2.2.ipynb).

In [11]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representa a la matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    # Nota: Esta función estima la SVD thin,la cual calcula unicamente las m columnas de U correspondientes a los m renglones de V. De esta
    # manera las columnas restantes de U no son calculadas, provocando una mejora significativa en velocidad de ejecución comparada con la 
    # la Full SVD. Referencia: https://en.wikipedia.org/wiki/Singular_value_decomposition#Thin_SVD.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicializa valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0
    stop<-FALSE
    
    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
    for(i in 1:(n-1)){
    for(j in (i+1):n){
      col_j<-ak[,j]
      col_i<-ak[,i]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-col_i%*%col_i
        b<-col_j%*%col_j
        c<-col_i%*%col_j
        
        #si c es cercano a cero no actualiza
        if(c<TOL){
            stop<-TRUE
            break}
          
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        temp<-ak[,i] 
        ak[,i]<-c(cs)*temp-c(sn)*ak[,j]
        ak[,j]<-c(sn)*temp+c(cs)*ak[,j]
        
        
        #actualiza las columnas de la matriz vk
        temp<-vk[,i] #cambio
        vk[,i]<-c(cs)*temp-c(sn)*vk[,j]
        vk[,j]<-c(sn)*temp+c(cs)*vk[,j]             
       }#cierra else
        }#cierra for j
      if(stop==TRUE){
        stop<-FALSE
        break
         }
        }#cierra for i
  k<-k+1
 }#cierra while
    
    #Obtener sigma
    sig<-apply(ak, 2, function(x){norm(x,"2")})

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            uk[,i]<-0  
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }   

### 3.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm

**Propósito:**

Dada una matriz $$A \in \mathbb{R}^{ m \times n}$$ y un vector $b \in \mathbb{R}^{ m}$, crear una función *sel_solver* que permita obtener aproximaciones numéricas asociadas a la descomposición SVD de ésta. En concreto, si la descomposición de $A$ está dada por las consabidas matrices $U \in \mathbb{R}^{m \times m}$, $\Sigma\in \mathbb{R}^{n \times n}$ y $V \in \mathbb{R}^{n \times n}$, emplear esta descomposición para resolver el sistema lineal según el procedimiento descrito en la sección 1.2.2

Ello se debe realizar con base en las siguientes especificaciones, la sección 1.2.2 y las implementaciones previas:

**Entradas:**

* $A \in \mathbb{R}^{ m \times n}$, 
* $b \in \mathbb{R}^{ m}$
* TOL (numero real, parámetro de tolerancia prefijado en $10^{-8}$),
* *maxsweeps* (entero mayor a cero). **Nota:** Las rotaciones que se aplicarán sucesivamente para ortogonalizar las matrices a partir de A se realizan en una secuencia con nombre *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n+1)}{2}$ rotaciones (pues depende de cuántas columnas son o no ortogonales) y en cada *sweep* se ortogonalizan 2 columnas. El número de *sweeps* a realizar se controla con esta variable.

**Salidas:**

* $x \in \mathbb{R}^{n}$ aproximación del sistema lineal $Ax=b$

**Funciones del proyecto  que depende:**

$svd_jacobi_aprox$ (véanse sección 3.1)

**Reporte de revisión:**

El reporte de revisión correspondiente a esta función se encuentra disponible para su consulta en este [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_Solver_SVD.ipynb)

In [8]:
sel_solver<-function(A,b,TOL=10**-8,maxsweep=20){
    # Aproxima la solución de un sistema de ecuaciones lineales (SEL) de la forma Ax=b 
    # usando la descomposición SVD de A obtenido por medio del método de One-sided Jacobi.
    # Args: 
    #    A (matriz): matriz de coeficientes del SEL, de números reales y de dimension m x n.
    #    b (vector): vector de lado derecho del sistema, de números reales y de dimension m.
    #    TOL (numeric): real positivo, que sirve para controlar la convergencia del metodo iterativo. 
    #             Nota: se usa un valor default de 10^-8 como tolerancia
    #    maxsweep (int): numero entero posiitivo, que se usar para controlar el número máximo de sweeps
    #                    que se ejecutan como parte del metodo iterativo.
    # Returns: 
    #    x (vector): vector solución del SEL, con dimension n.

    svd<-svd_jacobi_aprox(A,TOL,maxsweep)
    x<-solver(svd$U,svd$S,svd$V,b)
    x
}

### 3.3 Eliminación por bloques basada en solver linear que usa SVD

#### 3.3.1 Función de creación de bloques

**Propósito:**

Dada una matriz $A \in \mathbb{R}^{ m \times n}$ y un vector $b \in \mathbb{R}^{ m}$, crear una división en bloques de los mismo, de acuerdo a un parámetros que sirva para definir las dimensiones de sus primeros bloques $A_{11}$ y $b_1$, y en consecuencia el resto

**Entradas:**

* $A \in \mathbb{R}^{ m \times n}$, 
* $b \in \mathbb{R}^{ m}$
* TOL (numero real, parámetro de tolerancia prefijado en $10^{-8}$),


**Salidas:**

* Divisón de $A$ y $b$, en una lista de bloques $A11,A12, A21, A22$ y el vector $b$ dividido en 2 bloques $b1, b2$.

**Funciones del proyecto  que depende:**

Ninguna.

**Reporte de revisión:**

El reporte de revisión correspondiente a éste código se presente a través del siguiente [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_EliminacionXBloques_SVD.ipynb)

In [11]:
bloques<-function(A,b,corte) {
  # Dadas matriz A de n x ny un vector b de dimension n, así como un parametro corte (de dimension), 
  # genera una descomposicion en 4 y 2 bloques de estos.
  # En concreto, el parametro corte sirve para dividir A en los bloques de las siguientes dimensiones
  #    A11: bloque dado por A[1:corte,1:corte] (cuadrante superior izquierdo de A)
  #    A12: bloque dado por A[1:corte, corte+1:n] (cuadrante superior derecho de A)
  #    A21: bloque dado por A[corte+1:n,1:corte] (cuadrante inferior izquierdo de A)
  #    A22: bloque dado por A[corte+1:n,corte+1:n] (cuadrante inferior derecho de A)
  # Asimismo, b se divide como:
  #    b1: primeras corte-entradas de b
  #    b2: entradas restantes de b
    
  # Args: 
  #    A (matriz): matriz de dimensiones n x n sobre la que se hara la division en bloques, 
  #    b (vector): vector de dimensiones n sobre el cual que se hara la division en bloques,
  #    corte (int): entero positivo, que se usa como parametro de dimension con respecto al cual se hace la division de A (valor entre 1 y n)
  #Returns: 
  #     lista con la matriz A dividida en 4 bloques (lista): A11,A12, A21, A22 y el vector b dividido en 2 bloques b1, b2
  #     Notas: dimensiones de los bloques 
  #       A11 (corte x corte), A12 (corte x n-corte), A11 (n-corte x corte), A22 (n-corte x n-corte)
  #       b1 (corte) y b2 (n-corte)
    
    
  # Obtiene la descomposicion en bloques de A, segun parametro corte
  a11 <- A[c(1:corte),c(1:corte)]
  a12 <- A[c(1:corte),c((corte+1):dim(A)[2])]
  a21 <- A[c((corte+1):dim(A)[1]),c(1:corte)]
  a22 <- A[c((corte+1):dim(A)[1]),c((corte+1):dim(A)[2])]
    
  # Obtiene la descomposicion en bloques de b, segun parametro corte
  b1 <- b[1:(corte)]
  b2 <- b[((corte)+1):length(b)]

  list(A11 = a11,A12 =a12, A21=a21, A22=a22, b1 = b1, b2 = b2)
} 

#### 3.3.2 Solver de sistemas lineales usando método de eliminación por bloques y solver basado en Algoritmo de Jacobi One-Sided

**Propósito:**
 
Dada una matriz $A \in \mathbb{R}^{ m \times n}$, crear una función que permita obtener aproximaciones numéricas asociadas a la descomposición usando el método de eliminación por bloques, empleando la descomposición SVD asociada a los sistemas lineales inducidos por éstos. Ello se debe realizar con base en la sección 1.2.3:

**Entradas:**

* $A \in \mathbb{R}^{ m \times n}$,
* TOL (numero real, parámetro de tolerancia prefijado en $10^{-8}$),
* *maxsweeps* (entero mayor a cero). **Nota:** Las rotaciones que se aplicarán para el algortimo One-sided Jacobi para obtener la descomposición SVD de la matriz del sistema, sucesivamente para ortogonalizar las matrices a partir de A se realizan en una secuencia con nombre *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n+1)}{2}$ rotaciones (pues depende de cuántas columnas son o no ortogonales) y en cada *sweep* se ortogonalizan 2 columnas. El número de *sweeps* a realizar se controla con esta variable.

**Salidas:**

* $x \in \mathbb{R}^{m}$ solución del sistema lineal $Ax=b$.

**Funciones del proyecto  que depende:**

Todas las anteriores

**Reporte de revisión:**

El reporte de revisión correspondiente a éste código se presente a través del siguiente [vínculo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_EliminacionXBloques_SVD.ipynb)

In [13]:
install.packages("matrixcalc")
library("matrixcalc")


The downloaded binary packages are in
	/var/folders/4h/pz7sf1h93mn0vsyplpp3jxx40000gn/T//RtmpNuUwE1/downloaded_packages


In [14]:
eliminacion_bloques <- function(A,b,corte, TOL, maxsweep){
  # Aproxima la solución de un sistema de ecuaciones lineales (SEL) de la forma Ax=b, usando el método
  # de eliminación por bloques. La solución de los sistemas lineales inducidos por el complemento de Schur
  # se aproxima usando la descomposicion SVD aproximada con el algoritmo One-Sided Jacobi
  #Args: 
  #    A (matriz):  matriz de dimensiones n x n, asociada al sistema del sistema Ax=b que se dividira en bloques para su solucion
  #          Nota: para que exista solucion A y el el bloque A11 deben ser no singulares
  #    b (vector): vector de dimension n, asociado al sistema del sistema Ax=b que se dividira en bloques para su solucion
  #    corte (int): entero positivo, que se usara como parametro para divir en bloques a A y B, (valor entre 1 y n)
  #    TOL (numeric): real positivo, que sirve para controlar la convergencia del algoritmo One-Sided Jacobi para estimar SVD
  #    maxsweep (int): numero entero positivo, que sirve como parametro del máximo de sweeps del algoritmo One-Sided Jacobi
  #                    para estimar SVD (valor entre 1 y (n-1)*n/2)
  #Returns: 
  #    x: vector de dimensiones n x n

  # Descomposicion en bloques A11, A12, A21, A22, b1 y b2, segun parametro de dimension
  bloq = bloques(A,b,corte)

  # Evalua la singularidad de A y el bloque A11, en caso de que una de tales sea singular
  # arroja mensaje al usuario, en caso contrario intenta aproximar sistema inducidos por 
  # complemento de Schur
  if(is.singular.matrix(A,TOL) ==FALSE & is.singular.matrix(bloq$A11,TOL) == FALSE){

  # Solucion del sistema A11 y = b1, usando algoritmo One-Sided Jacobi
  y = sel_solver(bloq$A11,bloq$b1,TOL, maxsweep)
      
  # Solucion del sistema A11 Y = A12 
  Y= bloq$A12
  for(i in 1:dim(bloq$A12)[2]){
    Y[,i] = sel_solver(bloq$A11,bloq$A12[,i],TOL, maxsweep)
  }
      
  # Construye matrix S = A22 A11^-1 A12
  S = bloq$A22 - bloq$A21%*%Y

 # Construye b_hat = b2 - A21 A11^-1 b1
  b_hat = bloq$b2-bloq$A21%*%y
      
  # Solucion del sistema S x2 = b_hat algoritmo One-Sided Jacobi
  x_2 = sel_solver(S,b_hat,TOL, maxsweep)

  # Construye b_hat2 S x2 = b1 - A12 b_hat
  b_hat2 = bloq$b1 -bloq$A12%*%x_2

  # Solucion del sistema A11 x2 = b_hat algoritmo One-Sided Jacobi
  x_1 = sel_solver(bloq$A11,b_hat2,TOL, maxsweep)
      
  # Solucion del sistema original
  x <- c(x_1,x_2)
  x
  }else{
    print("Singularidad de las matrices involucradas; no hay solucion")
  }
  
}